## LOGISTIQUE REGRESSION

In [1]:
#import our librairies
import numpy as np
import pandas as pd
import numpy.linalg as LA
from sklearn.feature_extraction.text import CountVectorizer

## LOAD OUR DATA SET
##  as we have two dataset we have to caoncatenate its first

In [2]:
# import our dataset
data1 = pd.read_table('drugLibTest_raw.tsv',sep='\t')
data2 = pd.read_table('drugLibTrain_raw.tsv',sep='\t')
data = pd.concat((data1,data2), axis=0)
data.head()

,Unnamed: 0,urlDrugName,rating,effectiveness,sideEffects,condition,benefitsReview,sideEffectsReview,commentsReview
0,1366,biaxin,9,Considerably Effective,Mild Side Effects,sinus infection,The antibiotic may have destroyed bacteria cau...,"Some back pain, some nauseau.",Took the antibiotics for 14 days. Sinus infect...
1,3724,lamictal,9,Highly Effective,Mild Side Effects,bipolar disorder,Lamictal stabilized my serious mood swings. On...,"Drowsiness, a bit of mental numbness. If you t...",Severe mood swings between hypomania and depre...
2,3824,depakene,4,Moderately Effective,Severe Side Effects,bipolar disorder,Initial benefits were comparable to the brand ...,"Depakene has a very thin coating, which caused...",Depakote was prescribed to me by a Kaiser psyc...
3,969,sarafem,10,Highly Effective,No Side Effects,bi-polar / anxiety,It controlls my mood swings. It helps me think...,I didnt really notice any side effects.,This drug may not be for everyone but its wond...
4,696,accutane,10,Highly Effective,Mild Side Effects,nodular acne,Within one week of treatment superficial acne ...,Side effects included moderate to severe dry s...,Drug was taken in gelatin tablet at 0.5 mg per...


then i will group the data according to the length, 
and for that I will use dummies to convert my strings values in
numerical values for the data having a small length and countvectorize for the data having
a long length for the conversion

In [25]:
d1 = data[['effectiveness','sideEffects']]
#data['benefitsReview','commentsReview']= data['benefitsReview','commentsReview'].str.lower()
text= data[['benefitsReview','commentsReview']].astype(str).apply(lambda z: ' '.join(z),axis=1)
text = text.str.lower()
texte= text.replace('[\d+]')#remove numbers
d2 = texte
data3 = pd.get_dummies(d1)
 #list of text documents
text = d2
# create the transform
vectorizer = CountVectorizer( binary = True ,min_df=50,
                            stop_words ='english', max_features =50,
                                                         )
# tokenize and build vocab
tati = vectorizer.fit_transform(text)
data_text=tati.toarray()

 after that i transform it into dataframe and concatenate the both

In [26]:
data_text = pd.DataFrame(data_text)
D = pd.concat([pd.DataFrame(data3.values),pd.DataFrame(data_text.values)],axis=1)
D

,0,1,2,3,4,5,6,7,8,9,...,40,41,42,43,44,45,46,47,48,49
0,1,0,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,1,0,0,0,...,1,0,0,0,1,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,1,...,0,0,0,0,0,0,0,1,0,1
3,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,0,1,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4138,0,1,0,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,0,0
4139,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
4140,0,0,0,1,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4141,1,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,1,0,0,0,0


Define my features and target. here i use the whole dataset ie 100%

In [27]:
data['rating'] =data['rating'].apply(lambda x  :x-1)
y=data['rating'].values
x = D.values

## Define the function

In [28]:
class LogistiqueRegression(object):
    
    '''
    the logistique regression algorithm
    '''
    #initialise our variables
    def __init__(self,lr,n_iters,weight,x,y):
        self.lr = lr
        self.n_iters = n_iters
        self.weight = weight
        self.x = x
        self.y = y
      
    #the softmax function
    def Softmax(self,z):
        
        res = np.exp(z.T) / np.sum(np.exp(z), axis=1).T
        return res.T
    
    #cost function
    def cost(self):
        soft = self.Softmax(z)
        cost = 0
        for i in range(len(self.y)):
            for k in range(len(classes)):
                if self.y[i] == k:
                    cost+= -np.log(soft[i,k])
        return cost
    
    #the gradient
    def gradient(self):
        classes = np.unique(self.y)
        soft = self.Softmax(z)
        gradien = np.zeros((len(classes),self.x.shape[1]))
        
        for i in range(len(classes)):
            sume=0
            for k in range(self.x.shape[0]):
                if y[k] == i:
                    sume+=  x[i]*(1-soft[k,i]) 
                else:
                    sume+= x[i]*(-soft[k,i])
            gradien[i] = sume
        return -gradien
      
        
    def fit(self):
        classes = np.unique(self.y)
        grad = self.gradient()
        for i in range(len(classes)):
            for j in range(self.n_iters):
                self.weight[:,i] = self.weight[:,i] - self.lr * grad[i]
               # print(self.weight.shape)
        return self.weight 
    
    #the prediction
    def prediction(self,t):
        return np.argmax(self.Softmax(np.dot(t,self.fit())),axis=1)

### the Softmax function

In [29]:
classes = np.unique(y)
weight = np.zeros((x.shape[1], len(classes)))
z=np.dot(x,weight)
res = LogistiqueRegression(0.001,100,weight,x,y).Softmax(z)
res

array([[0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       ...,
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1]])

### the cost function

In [30]:
cost = LogistiqueRegression(0.001,100,weight,x,y).cost()
cost

8572.52430121726

### the gradient 

In [31]:
gradient = LogistiqueRegression(0.001,100,weight,x,y).gradient()
gradient

array([[ 278.3,   -0. ,   -0. ,   -0. ,   -0. ,   -0. ,  278.3,   -0. ,
          -0. ,   -0. ,   -0. ,   -0. ,   -0. ,   -0. ,  278.3,  278.3,
          -0. ,   -0. ,   -0. ,   -0. ,   -0. ,   -0. ,   -0. ,   -0. ,
          -0. ,   -0. ,   -0. ,   -0. ,   -0. ,   -0. ,   -0. ,   -0. ,
          -0. ,   -0. ,   -0. ,   -0. ,   -0. ,   -0. ,   -0. ,   -0. ,
          -0. ,   -0. ,   -0. ,   -0. ,   -0. ,   -0. ,   -0. ,   -0. ,
          -0. ,   -0. ,   -0. ,  278.3,   -0. ,   -0. ,   -0. ,   -0. ,
          -0. ,   -0. ,   -0. ,   -0. ],
       [  -0. ,  219.3,   -0. ,   -0. ,   -0. ,   -0. ,  219.3,   -0. ,
          -0. ,   -0. ,  219.3,   -0. ,   -0. ,   -0. ,  219.3,   -0. ,
         219.3,   -0. ,   -0. ,  219.3,   -0. ,   -0. ,   -0. ,   -0. ,
          -0. ,   -0. ,   -0. ,   -0. ,   -0. ,   -0. ,   -0. ,   -0. ,
          -0. ,   -0. ,  219.3,   -0. ,  219.3,   -0. ,   -0. ,   -0. ,
          -0. ,  219.3,   -0. ,  219.3,  219.3,   -0. ,   -0. ,   -0. ,
         219.3,  219.3,

### the prediction

In [32]:
X_train = x
X_test = x
Y_train = y
Y_test = y
d=LogistiqueRegression(0.001,100,weight,x,y).fit()

In [33]:
f=LogistiqueRegression(0.001,100,weight,x,y).prediction(X_test)
f

/usr/lib/python3/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in exp
/usr/lib/python3/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


array([8, 8, 8, ..., 6, 8, 8])

### the accuracy

In [34]:
(np.sum(f.reshape(1,-1) == Y_test.reshape(1,-1))/len(Y_test))*100

25.271542360608258

### Using 10% of the data

In [35]:
percentage = int(len(x)* 10/100)
x = x[:percentage,]
y = y[:percentage]
X_train = x[20:]
X_test = x[:20]
Y_train = y[20:]
Y_test = y[:20]

In [36]:
res = LogistiqueRegression(0.001,100,weight,x,y).Softmax(z)
cost = LogistiqueRegression(0.001,100,weight,x,y).cost()
gradient = LogistiqueRegression(0.001,100,weight,x,y).gradient()
d=LogistiqueRegression(0.001,100,weight,x,y).fit()

In [37]:
f=LogistiqueRegression(0.001,100,weight,x,y).prediction(X_test)
f

/usr/lib/python3/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in exp
/usr/lib/python3/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


array([8, 8, 8, 8, 8, 8, 6, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8])

In [38]:
(np.sum(f.reshape(1,-1) == Y_test.reshape(1,-1))/len(Y_test))*100

30.0

### Using 30% of the data

In [39]:
percentage = int(len(x)* 30/100)
x = x[:percentage,]
y = y[:percentage]
X_train = x[20:]
X_test = x[:20]
Y_train = y[20:]
Y_test = y[:20]

In [40]:
res = LogistiqueRegression(0.001,100,weight,x,y).Softmax(z)
cost = LogistiqueRegression(0.001,100,weight,x,y).cost()
gradient = LogistiqueRegression(0.001,100,weight,x,y).gradient()
d=LogistiqueRegression(0.001,100,weight,x,y).fit()

In [41]:
f=LogistiqueRegression(0.001,100,weight,x,y).prediction(X_test)
f

/usr/lib/python3/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in exp
/usr/lib/python3/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


array([8, 8, 8, 8, 8, 8, 6, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8])

In [42]:
(np.sum(f.reshape(1,-1) == Y_test.reshape(1,-1))/len(Y_test))*100

30.0

### Using 60% of the data

In [43]:
percentage = int(len(x)* 60/100)
x = x[:percentage,]
y = y[:percentage]
X_train = x[20:]
X_test = x[:20]
Y_train = y[20:]
Y_test = y[:20]

In [44]:
res = LogistiqueRegression(0.001,100,weight,x,y).Softmax(z)
cost = LogistiqueRegression(0.001,100,weight,x,y).cost()
gradient = LogistiqueRegression(0.001,100,weight,x,y).gradient()
d=LogistiqueRegression(0.001,100,weight,x,y).fit()

In [45]:
f=LogistiqueRegression(0.001,100,weight,x,y).prediction(X_test)
f

/usr/lib/python3/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: overflow encountered in exp
/usr/lib/python3/dist-packages/ipykernel_launcher.py:17: RuntimeWarning: invalid value encountered in true_divide


array([8, 8, 8, 8, 8, 8, 6, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8])

In [46]:
(np.sum(f.reshape(1,-1) == Y_test.reshape(1,-1))/len(Y_test))*100

30.0

## Observations

we note that based on the accuracy and the prediction, using all the data we we do not have a better perfomance of the model.So performance increases when data are reduced. We can conclude that the Logistique regression model is appropriate for this type of data